In [7]:
import numpy as np

# global variables
tree_score = None
nodes = {}
clusters = {}
mutations = {}
node_freq = {}
avg_obs_freq = {}

with open('example_output.txt', 'r') as file:
    cur_section = None
    for line in file:
        elements = line.strip().split() 
        if elements == ['TREE_SCORE']:
            cur_section = 'tree_score'
        elif elements == ['PARENT_NODE', 'CHILD_NODE']:
            cur_section = 'node'
        elif elements == ['CLUSTER', 'NODE_OF_CLUSTER']:
            cur_section = 'cluster'
        elif elements == ['MUTATION', 'NODE_OF_MUTATION']:
            cur_section = 'mutation'
        elif elements == ['NODE', 'NODE_FREQS', 'CLADE_FREQS']:
            cur_section = 'node_freq'
        elif elements == ['MUT_ID', 'OBSERVED_FREQS', 'INFERRED_FREQS', 'ABS_OBSERVED_MINUS_INFERRED']:
            cur_section = 'avg_obs_freq'
        elif elements:
            if cur_section == 'tree_score':
                tree_score = elements[0]
            elif cur_section == 'node':
                if int(elements[0]) not in nodes:
                    nodes[int(elements[0])] = set()
                nodes[int(elements[0])].add(int(elements[1]))
            elif cur_section == 'cluster':
                clusters[int(elements[1])] = int(elements[0])
            elif cur_section == 'mutation':
                mutations[elements[0]] = int(elements[1])
            elif cur_section == 'node_freq':
                node_freq[int(elements[0])] = [float(num) for num in elements[1].split(',')]
            elif cur_section == 'avg_obs_freq':
                if mutations[elements[0]] not in avg_obs_freq:
                    avg_obs_freq[mutations[elements[0]]] = []
                avg_obs_freq[mutations[elements[0]]].append([float(num) for num in elements[1].split(',')])

            
    for k, v in avg_obs_freq.items():
        avg_obs_freq[k] = [round(num, 4) for num in list(np.mean(v, axis = 0))]
    
    num_of_mutations = {}
    mutations_at_node = {}
    for k, v in mutations.items():
        if v not in num_of_mutations:
            num_of_mutations[v] = 1
            mutations_at_node[v] = [k]
        else:
            num_of_mutations[v] += 1
            mutations_at_node[v].append(k)

    print(mutations_at_node)

{6: ['chr10:35190958', 'chr12:22509130', 'chr13:71027393', 'chr14:100184987', 'chr16:55086544', 'chr16:758231', 'chr1:16635027', 'chr20:29059469', 'chr21:10445925', 'chr22:21555351', 'chr2:168253884', 'chr2:61789950', 'chr4:118384421', 'chr4:78253573', 'chr5:143910268', 'chr6:150131673', 'chr6:32554431', 'chr6:32554456', 'chr7:22030019', 'chr7:77473226', 'chr8:144993590', 'chr8:1858293', 'chr8:48021844', 'chr9:33367981', 'chr9:509285', 'chr9:91742118'], 7: ['chr11:2385519', 'chr17:41424507', 'chr19:7451021', 'chr21:10329899', 'chr21:10417801', 'chr21:9246746', 'chr22:10744644', 'chr22:11057057', 'chr6:10447082', 'chr7:100973146', 'chr7:100973185', 'chr7:100973191', 'chr7:100973217', 'chr7:100973224', 'chr7:152403470', 'chr9:33797664', 'chr9:33797665', 'chr9:65389815', 'chr9:65389841', 'chr9:66184841'], 4: ['chr12:111649006', 'chr17:67682883', 'chr20:33795887', 'chr2:20505119', 'chr2:20505129', 'chr3:125420133', 'chr3:183579769', 'chr3:35505119', 'chr3:64124514', 'chr4:188270148', 'chr4

In [8]:
import json

# create the json string

json_string = {}
def create_json(cur, node):
    cur["node"] = node
    cur["cluster"] = clusters[node]
    cur["mutations"] = num_of_mutations[node]
    cur["node_freq"] = node_freq[node]
    cur["avg_obs_freq"] = avg_obs_freq[node]
    cur["mutation_names"] = mutations_at_node[node]

    if node in nodes and nodes[node]:
        cur["children"] = []
        for i in nodes[node]:
            child = {}
            create_json(child, i)
            cur["children"].append(child)

create_json(json_string, 1)

filename = 'data.json'
with open(filename, "w") as f:
    json.dump(json_string, f, **{"indent": 4})